<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_2_SLSTR_file_structure.ipynb">Understanding SLSTR product structure >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2024 European Union <br>
**License:** MIT <br>
**Authors:** Ben Loveday (EUMETSAT/Innoflair UG), Hayley Evers-King (EUMETSAT)

<html>
   <div style="float:left"><p>&emsp;</p></div>
   <div style="float:left"><a href="https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/sensors/learn-slstr"><img src="https://img.shields.io/badge/open-EUMETLAB-E67E22.svg"></a></div>
   <div style="float:left"><p>&emsp;</p></div>
   <div style="float:left"><a href="https://user.eumetsat.int/data/themes/marine"><img src="https://img.shields.io/badge/open-USER PORTAL-154360.svg"></a></div>
   <div style="float:left"><p>&emsp;</p></div>
   <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fsensors%2Flearn-slstr/HEAD?labpath=1_SLSTR_introductory%2F1_1a_SLSTR_data_access_Data_Store.ipynb"><img src="https://mybinder.org/badge_logo.svg"></a></div>
   <div style="float:left"><p>&emsp;</p></div>
   <div style="float:left"><a href="https://jupyterhub.prod.wekeo2.eu/hub/user-redirect/lab/tree/public/wekeo4oceans/learn-slstr/1_SLSTR_introductory/1_1a_SLSTR_data_access_Data_Store.ipynb"><img src="https://img.shields.io/badge/launch-WEKEO-1a4696.svg"></a></div>
   <div style="float:left"><p>&emsp;</p></div>
   <div style="float:left"><a href="https://code.insula.destine.eu/hub/"><img src="https://img.shields.io/badge/launch-DestinE-f43fd3.svg"></a></div></div>
</html>

<div class="alert alert-block alert-success">
<h3>Learn SLSTR: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **<a href="https://eoportal.eumetsat.int/" target="_blank">A EUMETSAT Earth Observation Portal account</a>** if you are using or plan to use the EUMETSAT Data Store or CODA

There are no prerequisite notebooks for this module.
</div>
<hr>

# 1.1 Accessing SLSTR data via the EUMETSAT Data Store

### Data used

| Dataset | EUMETSAT collection ID| EUMETSAT collection<br>description | WEkEO dataset ID | WEkEO description |
|:--------------------:|:-----------------------:|:-------------:|:-----------------:|:-----------------:|
| Sentinel-3 SLSTR level 1b | EO:EUM:DAT:0411 | <a href="https://user.eumetsat.int/catalogue/EO:EUM:DAT:SENTINEL-3:SL_1_RBT___NTC" target="_blank">Description</a> | EO:EUM:DAT:SENTINEL-3:SL_1_RBT___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASL_1_RBT___" target="_blank">Description</a> |
| Sentinel-3 SLSTR level 2 | EO:EUM:DAT:0412 | <a href="https://user.eumetsat.int/catalogue/EO:EUM:DAT:SENTINEL-3:SL_2_WST___NTC" target="_blank">Description</a> | EO:EUM:DAT:SENTINEL-3:SL_2_WST___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASL_2_WST___" target="_blank">Description</a> |

### Learning outcomes

At the end of this notebook you will know;
* how to download data from the EUMETSAT Data Store using the <font color="#138D75">**eumetsat data access (eumdac)**</font> client

### Outline

Data from SLSTR is available through multiple sources, either via a web user interface (WebUI) or through code and command line interfaces with an Application Programming Interface (API). WebUIs are useful for accessing quick-look data visualisations, and for browsing to see what is available. APIs are more useful for routine, automated and operational data access. Here we will guide you through ways you can access SLSTR data through both methods.

Users should note that this notebook will use the [Data Store APIs](#section3) to download <font color="#138D75">**all**</font> of the products required for the lessons in the SLSTR introductory module. If you were working on a local machine, using SLSTR and other data served directly by the EUMETSAT Data Store, this should be your preferred access method. Users working within the WEkEO environment (i.e. on a WEkEO JupyterHub or virtual machine) should follow the instructions in the companion notebook, <a href="./1_1b_SLSTR_data_access_HDA.ipynb">Accessing SLSTR data via the WEKEO HDA adaptor</a>, as this will be faster.

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Download data via GUIs](#section1)
 1. [The products we need](#section2)
 1. [Downloading via the Data Store API](#section3)
 1. [Downloading via the WEkEO API](#section4)

<hr>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [ ]:
import configparser      # a library that allows us to parse standard configuration files
import IPython           # a library that helps us display video and HTML content
import os                # a library that allows us access to basic operating system commands like making directories
import shutil            # a library that allows us access to basic operating system commands like copy
import zipfile           # a library that allows us to unzip zip-files.
import eumdac            # a tool that helps us download via the eumetsat/data-store
from pathlib import Path # a library that helps construct system path objects
import getpass           # a library to help us enter passwords

Sometimes we use configuration files to help us set some notebook parameters. The box below reads a configuration file to help us decide how large to make the videos displayed below

In [ ]:
# set defaults and overwrite with frameworks config if it exists
config = configparser.ConfigParser()
config["nbook"] = {"v_wd" : "700", "v_ht" : "450"}
if os.path.exists(os.path.join(os.path.dirname(os.getcwd()),"frameworks","config.ini")):
    config.read(os.path.join(os.path.dirname(os.getcwd()),"frameworks","config.ini"))

Next we will create a download directory to store the products we will download in this notebook.

In [ ]:
# Create a download directory for our SLSTR products
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Downloading data via GUIs
[Back to top](#TOC_TOP)

</div>

### The EUMETSAT Data Store

The <a href="https://data.eumetsat.int" target="_blank">Data Store</a> is EUMETSAT's primary pull service for delivering data, including the ocean data available from Sentinel-3 and SLSTR. 

Access to it is possible through a WebUI, and through a series of application programming interfaces (APIs). The Data Store supports browsing, searching and downloading data as well as subscription services. It also provides a link to the online version of the 

<a href="https://tailor.eumetsat.int/" target="_blank">EUMETSAT Data Tailor</a> for customisation. The video below provides an overview of the Data Store WebUI.

The video below explains how you can access data through the EUMETSAT Data Store WebUI. You can visit and see if you can find SLSTR data to download, however this notebook will also show you how to access the Data Store API, facilitated by the EUMDAC client (see the [Downloading via the Data Store API](#section3) section, below).

#### Links:
* <a href="https://data.eumetsat.int" target="_blank">EUMETSAT Data Store</a>
* <a href="https://eumetsatspace.atlassian.net/wiki/spaces/DSDS/overview" target="_blank">More information on the Data Store</a>

In [ ]:
IPython.display.YouTubeVideo('WINakzAZvxw', width=config["nbook"]["v_wd"], height=config["nbook"]["v_ht"])

<div class="alert alert-info" role="alert">

## 2. <a id='section2'></a>The products we need for this module
[Back to top](#TOC_TOP)
    
</div>

We need a number of SLSTR level-1b and level-2 products for the notebooks within the introductory part of the learn-slstr module. These products are shown below in a python `list` called *product list*. We can retrieve these products from either the Data Store or the WEkEO product catalogue.

In [ ]:
product_list = [
'S3A_SL_1_RBT____20220209T225750_20220209T230050_20220211T073550_0179_082_001_3600_MAR_O_NT_004.SEN3',
'S3A_SL_1_RBT____20220209T220650_20220209T220950_20220211T045404_0179_082_001_0540_MAR_O_NT_004.SEN3',
'S3A_SL_2_WST____20220209T213232_20220209T231332_20220211T074946_6059_081_385______MAR_O_NT_003.SEN3'
]

<hr>
<div class="alert alert-info" role="alert">

## 3. <a id='section3'></a>Downloading from the Data Store via API
[Back to top](#TOC_TOP)

</div>

<div class="alert alert-block alert-success">

### Accessing the EUMETSAT Data Store

To access Copernicus marine data from the <a href="https://data.eumetsat.int " target="_blank">EUMETSAT Data Store</a>, we will use the EUMETSAT Data Access Client (`eumdac`) python package. If you are working with the recommended Anaconda Python distribution and used the environment file included in this repository (environment.yml) to build this python environment (as detailed in the README), you will already have installed this. If not, you can install eumdac using;

`conda install -c eumetsat eumdac`

You can also find the source code on the <a href="https://gitlab.eumetsat.int/eumetlab/data-services/eumdac " target="_blank">EUMETSAT GitLab</a>. Please visit the EUMETSAT user portal for more information on the <a href="https://user.eumetsat.int/data-access/data-store " target="_blank">EUMETSAT Data Store</a> and <a href="https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide " target="_blank">eumdac</a>.

To download data from the EUMETSDAT Data Store via API, you need to provide credentials. To obtain these you should first register at for an <a href="https://eoportal.eumetsat.int/" target="_blank">EUMETSAT Earth Observation Portal account</a>. Once you have an account, you can retrieve your `<your_consumer_key>` and `<your_consumer_secret>` from the <a href="https://api.eumetsat.int/api-key/ " target="_blank">"EUMETSAT Data Store API"</a> page (*Note: you must click the "Show hidden fields" button at the bottom of the page to see the relevant fields*). If you do not already have a local credentials file, you will be prompted to enter your credentials when you run the cell below. This will create the required local credentials file, so that you only need to run this once.

*Note: your key and secret are permanent, so you should take care to never share them*

</div>

In [ ]:
# load credentials
eumdac_credentials_file = Path(Path.home() / '.eumdac' / 'credentials')

if os.path.exists(eumdac_credentials_file):
    consumer_key, consumer_secret = Path(eumdac_credentials_file).read_text().split(',')
else:
    # creating authentication file
    consumer_key = input('Enter your consumer key: ')
    consumer_secret = getpass.getpass('Enter your consumer secret: ')
    try:
        os.makedirs(os.path.dirname(eumdac_credentials_file), exist_ok=True)
        with open(eumdac_credentials_file, "w") as f:
            f.write(f'{consumer_key},{consumer_secret}')
    except:
        pass
        
token = eumdac.AccessToken((consumer_key, consumer_secret))
print(f"This token '{token}' expires {token.expiration}")

Now we have a token, we can create an instance of the EUMETSAT Data Store.

In [ ]:
datastore = eumdac.DataStore(token)

Data in the EUMETSAT Data Store are stored as collections, each with its own ID. If we don't know our collection ID a priori, we can find this information via the Data Store GUI or by asking `eumdac` to tell us about all available relevant collections as follows:

In [ ]:
for collection_id in datastore.collections:
    if ("SLSTR" in collection_id.title):
        if "non-public" in collection_id.abstract: continue
        print(f"Collection ID({collection_id}): {collection_id.title}")

So, for;
* **SLSTR Level 1B Radiances and Brightness Temperatures** we want `collection_id`: **EO:EUM:DAT:0411**.
* **SLSTR Level 2 Sea Surface Temperature (SST)** we want `collection_id`: **EO:EUM:DAT:0412**

Lets get some level-1B data first. Below we provide the level-1B `collection_id` to our `datastore` object to choose the correct collection.

In [ ]:
collection_id = 'EO:EUM:DAT:0411'
selected_collection = datastore.get_collection(collection_id)

In this case, as we already know what product we want, we can now find the product directly using the **product name**. which in this case the first item in our product list. 

In [ ]:
selected_product = datastore.get_product(product_id=product_list[0], collection_id=collection_id)

For more advanced search options using EUMDAC see the notebook on <a href="../2_SLSTR_advanced/2_1_SLSTRI_advanced_data_access_eumdac.ipynb">advanced data access with eumdac</a>.

Now we can download the product. It will go into our download directory *(../products)*.

In [ ]:
with selected_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
    print(f'Downloading {fsrc.name}')
    shutil.copyfileobj(fsrc, fdst)
    print(f'Download of product {fsrc.name} finished.')

The product is downloaded as a zip file, so lets unzip it and remove the zip.

In [ ]:
with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.startswith(str(selected_product)):
            zip_ref.extract(file, download_dir)
    print(f'Unzipping of product {selected_product} finished.')
os.remove(fdst.name)

You can now check the **products** directory to see that we have the level 1B file. Now, lets pull all the parts together and download the second level-1 and the level 2 file in a single loop.

In [ ]:
collection_ids = ["EO:EUM:DAT:0411", "EO:EUM:DAT:0412"]

for product_id, collection_id in zip(product_list[1:],collection_ids):
    print(f"Retrieving: {product_id}")    

    selected_collection = datastore.get_collection(collection_id)
    selected_product = datastore.get_product(product_id=product_id, collection_id=collection_id)
    
    with selected_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
        print(f'Downloading {fsrc.name}.')
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of product {fsrc.name} finished.')

    with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.startswith(str(selected_product)):
                zip_ref.extract(file, download_dir)
        print(f'Unzipping of product {fdst.name} finished.')

    os.remove(fdst.name)

If you have completed this section, you can now continue to work through the rest of the notebooks in this repository and learn more about working with SLSTR data. Alternatively, if you'd like to learn more about using the advanced functionality of EUMDAC to access SLSTR data you can check out the advanced workflows in the <a href="../2_SLSTR_advanced/2_1_SLSTR_advanced_data_access_eumdac.ipynb">advanced data access with eumdac</a> notebook. 

<hr>
<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_2_SLSTR_file_structure.ipynb">Understanding SLSTR product structure >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean" target="_blank">View on GitLab</a> | <a href="https://training.eumetsat.int/" target="_blank">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int target="_blank">Contact helpdesk for support </a> | <a href=mailto:training@eumetsat.int target="_blank">Contact our training team to collaborate on and reuse this material</a></span></p>